## 1. 라이브러리 및 파일 합치기

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# 현재 작업 디렉터리 출력
current_directory = os.getcwd()
print("현재 작업 디렉터리:", current_directory)
# 종관기상관측 폴더 경로 설정
asos_folder_path = os.path.join('data', '종관기상관측(ASOS)기상데이터')
print("asos디렉터리:", asos_folder_path)
# 폴더 내 파일 리스트 가져오기
asos_files = os.listdir(asos_folder_path)
print(asos_files)

현재 작업 디렉터리: c:\Users\Hyuk\Documents\대학\4-1학기\3.소프트웨어융합프로젝트\산업통상자원부프로젝트
asos디렉터리: data\종관기상관측(ASOS)기상데이터
['1001_1912OBS_ASOS_DD.csv', '2001_2412OBS_ASOS_DD.csv', '가공후']


### 1-1 데이터 확인

In [ ]:

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

def imfomation_asos_data(dirPath):
    '''
    폴더 경로를 입력받아 폴더 내 모든 파일을 읽어오고, 각 파일의 데이터 기본 정보를 출력하는 함수
    '''
    # 데이터 파일 경로
    files = os.listdir(dirPath)
    print("파일 목록", files)
    for file in files:
        file_path = os.path.join(dirPath, file)
        print(file_path)
        # current_directory = os.getcwd()
        # full_file_path = os.path.join(current_directory, file_path)
        # 데이터 읽기   
        print("데이터를 읽는 중...")    
        df = pd.read_csv(file_path, encoding='cp949')
        # 데이터 기본 정보 출력
        print(f"\n======= {file} =======")
        print(f"\n=== 데이터 기본 정보 ===")
        # print("데이터 크기:", df.shape)
        print("\n컬럼 목록:")
        print(df.columns.tolist())
        print("\n처음 5개 행:")
        print(df.head())
        print("\n데이터 타입:")
        print(df.dtypes)
    
        # 결측치 확인
        print("\n=== 결측치 현황 ===")
        missing_values = df.isnull().sum()
        missing_percentages = (missing_values / len(df)) * 100
        print(missing_percentages)

imfomation_asos_data(asos_folder_path)

### 1-2. 파일 합치기

In [ ]:
asos_files = os.listdir(asos_folder_path)
dfs = []
sum = 0
for file in asos_files:
    df = pd.read_csv(os.path.join(asos_folder_path, file), encoding='cp949')
    dfs.append(df)
    sum += df.shape[0]
    print(f"{file}: ", df.shape)

df = pd.concat(dfs)
print(df.shape)
output_file = os.path.join(asos_folder_path, 'ASOS_통합.csv')
print(f"\n데이터를 {output_file}에 저장하는 중...")
df.to_csv(output_file, index=False, encoding='cp949')


## 2. 결측치

In [ ]:
df = pd.read_csv(os.path.join(asos_folder_path, 'ASOS_통합.csv'), encoding='cp949')

### 2-1 일사량 데이터의 결측률이 5% 이상일 때 제거

In [34]:
import pandas as pd
import os

# CSV 파일 경로 설정
file_path = os.path.join(os.path.join(asos_folder_path, '가공후'), 'ASOS_통합(1301~2412).csv')

# 지역 컬럼
location_column = '지점명'

# 일사량 관련 컬럼
col_hhmi = '1시간 최다일사 시각(hhmi)'        # Hour of maximum insolation
col_max = '1시간 최다일사량(MJ/m2)'          # Max insolation per hour
col_sum = '합계 일사량(MJ/m2)'               # Total insolation

# 날짜 컬럼 (연월일 컬럼 명 확인 필요, 가정: '일시')
date_column = '일시'

# 지역별 전체 건수 및 결측치 건수 기록용 dict
solar_stats = {}

# 데이터 청크 단위로 읽기
for chunk in pd.read_csv(file_path, encoding='cp949', chunksize=10000):
    # 날짜 형식으로 변환
    chunk[date_column] = pd.to_datetime(chunk[date_column], errors='coerce')
    
    # 2020-01-01 ~ 2024-12-31 범위로 필터링
    # chunk = chunk[
    #     (chunk[date_column] >= '2014-01-01') &
    #     (chunk[date_column] <= '2024-12-31')
    # ]
    
    # 필요한 컬럼만 추출
    chunk = chunk[[location_column, col_hhmi, col_max, col_sum]]
    
    # 그룹별 전체 행 수
    total = chunk.groupby(location_column).size()
    
    # 그룹별 결측치 수 계산
    missing = chunk.groupby(location_column)[[col_hhmi, col_max, col_sum]].apply(lambda df: df.isna().sum())
    
    for region in total.index:
        # 초기화
        if region not in solar_stats:
            solar_stats[region] = {
                'total': 0, 'missing': [0, 0, 0]
            }
        
        # 누적 전체 행 수
        solar_stats[region]['total'] += total[region]
        
        # 누적 결측치 수
        solar_stats[region]['missing'][0] += missing.loc[region, col_hhmi]
        solar_stats[region]['missing'][1] += missing.loc[region, col_max]
        solar_stats[region]['missing'][2] += missing.loc[region, col_sum]

# 최종 결측률 계산 후 20% 이상인 지역 찾기
excluded_regions = []

for region, stats in solar_stats.items():
    total_rows = stats['total']
    missing_counts = stats['missing']
    
    # 각 컬럼의 결측률 계산
    missing_ratios = [count / total_rows for count in missing_counts]
    
    # 3개 중 하나라도 20% 이상이면 제외 대상
    if any(ratio >= 0.05 for ratio in missing_ratios):
        excluded_regions.append(region)

# 결과 출력
print("❌ 삭제할 지역 목록 (결측률 ≥ 5%):", excluded_regions)
print(f"삭제 지역 수: {len(excluded_regions)} / 총 지역 수: {len(solar_stats)}")

# 상세 출력
for region, stats in solar_stats.items():
    print(f"{region}: 전체={stats['total']} | 결측수={stats['missing']} | 결측률={[round(v / stats['total'] * 100, 2) for v in stats['missing']]}")


❌ 삭제할 지역 목록 (결측률 ≥ 5%): ['동두천', '속초', '철원', '파주', '백령도', '동해', '영월', '울릉도', '울진', '충주', '상주', '군산', '울산', '창원', '여수', '통영', '순천', '완도', '진도(첨찰산)', '강화', '서귀포', '성산', '양평', '이천', '인제', '홍천', '보은', '정선군', '제천', '태백', '금산', '보령', '부여', '천안', '남원', '부안', '세종', '임실', '정읍', '장수', '고흥', '장흥', '해남', '문경', '봉화', '영주', '진도군', '구미', '영덕', '의성', '거창', '영천', '합천', '거제', '남해', '밀양', '산청']
삭제 지역 수: 57 / 총 지역 수: 99
동두천: 전체=4383 | 결측수=[np.int64(3205), np.int64(3205), np.int64(3205)] | 결측률=[np.float64(73.12), np.float64(73.12), np.float64(73.12)]
북춘천: 전체=3014 | 결측수=[np.int64(5), np.int64(5), np.int64(5)] | 결측률=[np.float64(0.17), np.float64(0.17), np.float64(0.17)]
속초: 전체=4383 | 결측수=[np.int64(4123), np.int64(4125), np.int64(4125)] | 결측률=[np.float64(94.07), np.float64(94.11), np.float64(94.11)]
철원: 전체=4383 | 결측수=[np.int64(2236), np.int64(2236), np.int64(2236)] | 결측률=[np.float64(51.02), np.float64(51.02), np.float64(51.02)]
파주: 전체=4383 | 결측수=[np.int64(3204), np.int64(3206), np.int64(3206)] | 결측률=[np.float6

In [35]:
# 원본 데이터 로드
input_file_path = os.path.join(os.path.join(asos_folder_path, '가공후'), 'ASOS_통합(1301~2412).csv')
print(f"Loading data from {input_file_path}...")
df_to_filter = pd.read_csv(input_file_path, encoding='cp949', low_memory=False)
print("Original shape:", df_to_filter.shape)

# excluded_regions에 포함된 지역 데이터 제거
print("Filtering out regions with high missing data...")
df_filtered = df_to_filter[~df_to_filter[location_column].isin(excluded_regions)]
print("Shape after filtering:", df_filtered.shape)

# 필터링된 데이터를 새 파일로 저장
output_file_path = os.path.join(os.path.join(asos_folder_path, '가공후'), 'ASOS_통합(1301~2412)_filtered.csv')
print(f"Saving filtered data to {output_file_path}...")
df_filtered.to_csv(output_file_path, index=False, encoding='cp949')
print("Saved successfully.")

Loading data from data\종관기상관측(ASOS)기상데이터\가공후\ASOS_통합(1301~2412).csv...
Original shape: (415497, 62)
Filtering out regions with high missing data...
Shape after filtering: (170482, 62)
Saving filtered data to data\종관기상관측(ASOS)기상데이터\가공후\ASOS_통합(1301~2412)_filtered.csv...
Saved successfully.


In [37]:
import pandas as pd
import os

# CSV 파일 경로 설정
file_path = os.path.join(os.path.join(asos_folder_path, '가공후'), 'ASOS_통합(1301~2412).csv')

# 지역 컬럼
location_column = '지점명'

# 일사량 관련 컬럼
col_hhmi = '1시간 최다일사 시각(hhmi)'        # Hour of maximum insolation
col_max = '1시간 최다일사량(MJ/m2)'          # Max insolation per hour
col_sum = '합계 일사량(MJ/m2)'               # Total insolation

# 날짜 컬럼 (연월일 컬럼 명 확인 필요, 가정: '일시')
date_column = '일시'

# 지역별 전체 건수 및 결측치 건수 기록용 dict
solar_stats = {}

# 데이터 청크 단위로 읽기
for chunk in pd.read_csv(file_path, encoding='cp949', chunksize=10000):
    # 날짜 형식으로 변환
    chunk[date_column] = pd.to_datetime(chunk[date_column], errors='coerce')
    
    # 2020-01-01 ~ 2024-12-31 범위로 필터링
    chunk = chunk[
        (chunk[date_column] >= '2020-01-01') &
        (chunk[date_column] <= '2024-12-31')
    ]
    
    # 필요한 컬럼만 추출
    chunk = chunk[[location_column, col_hhmi, col_max, col_sum]]
    
    # 그룹별 전체 행 수
    total = chunk.groupby(location_column).size()
    
    # 그룹별 결측치 수 계산
    missing = chunk.groupby(location_column)[[col_hhmi, col_max, col_sum]].apply(lambda df: df.isna().sum())
    
    for region in total.index:
        # 초기화
        if region not in solar_stats:
            solar_stats[region] = {
                'total': 0, 'missing': [0, 0, 0]
            }
        
        # 누적 전체 행 수
        solar_stats[region]['total'] += total[region]
        
        # 누적 결측치 수
        solar_stats[region]['missing'][0] += missing.loc[region, col_hhmi]
        solar_stats[region]['missing'][1] += missing.loc[region, col_max]
        solar_stats[region]['missing'][2] += missing.loc[region, col_sum]

# 최종 결측률 계산 후 20% 이상인 지역 찾기
excluded_regions = []

for region, stats in solar_stats.items():
    total_rows = stats['total']
    missing_counts = stats['missing']
    
    # 각 컬럼의 결측률 계산
    missing_ratios = [count / total_rows for count in missing_counts]
    
    # 3개 중 하나라도 20% 이상이면 제외 대상
    if any(ratio >= 0.05 for ratio in missing_ratios):
        excluded_regions.append(region)

# 결과 출력
print("❌ 삭제할 지역 목록 (결측률 ≥ 5%):", excluded_regions)
print(f"삭제 지역 수: {len(excluded_regions)} / 총 지역 수: {len(solar_stats)}")

# 원본 데이터 로드
input_file_path = os.path.join(os.path.join(asos_folder_path, '가공후'), 'ASOS_통합(1301~2412).csv')
print(f"Loading data from {input_file_path}...")
df_to_filter = pd.read_csv(input_file_path, encoding='cp949', low_memory=False)
print("Original shape:", df_to_filter.shape)

# excluded_regions에 포함된 지역 데이터 제거
print("Filtering out regions with high missing data...")
df_filtered = df_to_filter[~df_to_filter[location_column].isin(excluded_regions)]
print("Shape after filtering:", df_filtered.shape)

# 필터링된 데이터를 새 파일로 저장
output_file_path = os.path.join(os.path.join(asos_folder_path, '가공후'), 'ASOS_통합(2001~2412)_filtered.csv')
print(f"Saving filtered data to {output_file_path}...")
df_filtered.to_csv(output_file_path, index=False, encoding='cp949')
print("Saved successfully.")

❌ 삭제할 지역 목록 (결측률 ≥ 5%): ['동두천', '속초', '파주', '동해', '영월', '울진', '충주', '군산', '상주', '울산', '완도', '통영', '성산', '순천', '강화', '서귀포', '양평', '이천', '보은', '인제', '정선군', '제천', '태백', '홍천', '금산', '보령', '부여', '세종', '천안', '남원', '부안', '임실', '장수', '정읍', '고흥', '장흥', '해남', '봉화', '영주', '진도군', '구미', '문경', '영덕', '의성', '거창', '밀양', '산청', '영천', '합천', '거제', '남해']
삭제 지역 수: 51 / 총 지역 수: 97
Loading data from data\종관기상관측(ASOS)기상데이터\가공후\ASOS_통합(1301~2412).csv...
Original shape: (415497, 62)
Filtering out regions with high missing data...
Shape after filtering: (194806, 62)
Saving filtered data to data\종관기상관측(ASOS)기상데이터\가공후\ASOS_통합(2001~2412)_filtered.csv...
Saved successfully.


## 3. Regional grouping

In [ ]:
import pandas as pd
import os

# Setting the CSV file path
# 필터링된 CSV 파일 경로 설정
filtered_file_path = os.path.join(os.path.join(asos_folder_path, '가공후'), 'ASOS_통합_filtered.csv')

try:
    #Variable initialization
    # 데이터 처리를 위한 변수 초기화
    total_rows = 0
    chunk_list = []
    
    print(f"'{filtered_file_path}'에서 청크 단위로 데이터를 읽어옵니다...")

    # CSV 파일을 청크 단위로 순회ㅊ
    for i, chunk in enumerate(pd.read_csv(filtered_file_path, encoding='cp949', chunksize=10000, low_memory=False)):
        # 첫 번째 청크의 정보만 출력
        if i == 0:
            print("\n--- 첫 번째 청크 정보 ---")
            print(f"청크의 형태: {chunk.shape}")
            print("\n청크의 첫 5행:")
            print(chunk.head())
            print("\n청크의 데이터 타입:")
            print(chunk.dtypes)
            print("--------------------------\n")
        
        chunk_list.append(chunk)
        total_rows += len(chunk)

    # 모든 청크를 하나의 데이터프레임으로 병합
    if chunk_list:
        df_final = pd.concat(chunk_list, ignore_index=True)

        # 전체 데이터에 대한 통계 정보 출력
        print("\n--- 전체 데이터 통계 ---")
        print(f"처리된 총 행 수: {total_rows}")
        print(f"최종 데이터프레임 형태: {df_final.shape}")
        print("\n데이터프레임 정보:")
        df_final.info()
        print("\n기술 통계:")
        print(df_final.describe())
        print("--------------------------")
    else:
        print("필터링된 파일에 데이터가 없습니다.")

except FileNotFoundError:
    print(f"오류: '{filtered_file_path}'에서 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

'data\종관기상관측(ASOS)기상데이터\가공후\ASOS_통합_filtered.csv'에서 청크 단위로 데이터를 읽어옵니다...

--- 첫 번째 청크 정보 ---
청크의 형태: (10000, 62)

청크의 첫 5행:
   지점  지점명          일시  평균기온(°C)  최저기온(°C)  최저기온 시각(hhmi)  최고기온(°C)  \
0  93  북춘천  2016-10-01      18.6      14.7          538.0      25.3   
1  93  북춘천  2016-10-02      17.0      15.6          520.0      19.4   
2  93  북춘천  2016-10-03      20.5      15.0         2334.0      29.0   
3  93  북춘천  2016-10-04      18.3      11.4          619.0      26.3   
4  93  북춘천  2016-10-05      18.0      14.2         2339.0      25.6   

   최고기온 시각(hhmi)  강수 계속시간(hr)  10분 최다 강수량(mm)  ...  0.5m 지중온도(°C)  \
0         1515.0          NaN             NaN  ...           22.0   
1         1123.0        15.17             4.0  ...           22.0   
2         1557.0         8.50             8.5  ...           20.9   
3         1259.0          NaN             NaN  ...           21.5   
4         1527.0         1.83             0.0  ...           21.5   

   1.0m 지중온도(°C)  1.5m 지중온도(°C)  3

In [24]:
import pandas as pd
import os

# CSV 파일 경로 설정
file_path = os.path.join(os.path.join(asos_folder_path, '가공후'), 'ASOS_통합_filtered.csv')

# 지역 컬럼
location_column = '지점명'

# 일사량 관련 컬럼
col_hhmi = '1시간 최다일사 시각(hhmi)'        # Hour of maximum insolation
col_max = '1시간 최다일사량(MJ/m2)'          # Max insolation per hour
col_sum = '합계 일사량(MJ/m2)'               # Total insolation

# 날짜 컬럼 (연월일 컬럼 명 확인 필요, 가정: '일시')
date_column = '일시'

# 지역별 전체 건수 및 결측치 건수 기록용 dict
solar_stats = {}

# 데이터 청크 단위로 읽기
for chunk in pd.read_csv(file_path, encoding='cp949', chunksize=10000):
    # 날짜 형식으로 변환
    chunk[date_column] = pd.to_datetime(chunk[date_column], errors='coerce')
    
    # 2020-01-01 ~ 2024-12-31 범위로 필터링
    # chunk = chunk[
    #     (chunk[date_column] >= '2020-01-01') &
    #     (chunk[date_column] <= '2024-12-31')
    # ]
    
    # 필요한 컬럼만 추출
    chunk = chunk[[location_column, col_hhmi, col_max, col_sum]]
    
    # 그룹별 전체 행 수
    total = chunk.groupby(location_column).size()
    
    # 그룹별 결측치 수 계산
    missing = chunk.groupby(location_column)[[col_hhmi, col_max, col_sum]].apply(lambda df: df.isna().sum())
    
    for region in total.index:
        # 초기화
        if region not in solar_stats:
            solar_stats[region] = {
                'total': 0, 'missing': [0, 0, 0]
            }
        
        # 누적 전체 행 수
        solar_stats[region]['total'] += total[region]
        
        # 누적 결측치 수
        solar_stats[region]['missing'][0] += missing.loc[region, col_hhmi]
        solar_stats[region]['missing'][1] += missing.loc[region, col_max]
        solar_stats[region]['missing'][2] += missing.loc[region, col_sum]

# 최종 결측률 계산 후 20% 이상인 지역 찾기
excluded_regions = []

for region, stats in solar_stats.items():
    total_rows = stats['total']
    missing_counts = stats['missing']
    
    # 각 컬럼의 결측률 계산
    missing_ratios = [count / total_rows for count in missing_counts]
    
    # 3개 중 하나라도 20% 이상이면 제외 대상
    if any(ratio >= 0.01 for ratio in missing_ratios):
        excluded_regions.append(region)

# 결과 출력
print("❌ 삭제할 지역 목록 (결측률 ≥ 20%):", excluded_regions)
print(f"삭제 지역 수: {len(excluded_regions)} / 총 지역 수: {len(solar_stats)}")

# 상세 출력
for region, stats in solar_stats.items():
    print(f"{region}: 전체={stats['total']} | 결측수={stats['missing']} | 결측률={[round(v / stats['total'] * 100, 2) for v in stats['missing']]}")

❌ 삭제할 지역 목록 (결측률 ≥ 20%): []
삭제 지역 수: 0 / 총 지역 수: 37
강릉: 전체=4383 | 결측수=[np.int64(12), np.int64(12), np.int64(12)] | 결측률=[np.float64(0.27), np.float64(0.27), np.float64(0.27)]
대관령: 전체=4383 | 결측수=[np.int64(9), np.int64(9), np.int64(9)] | 결측률=[np.float64(0.21), np.float64(0.21), np.float64(0.21)]
북강릉: 전체=4383 | 결측수=[np.int64(6), np.int64(6), np.int64(6)] | 결측률=[np.float64(0.14), np.float64(0.14), np.float64(0.14)]
북춘천: 전체=3014 | 결측수=[np.int64(5), np.int64(5), np.int64(5)] | 결측률=[np.float64(0.17), np.float64(0.17), np.float64(0.17)]
춘천: 전체=4382 | 결측수=[np.int64(0), np.int64(1), np.int64(2)] | 결측률=[np.float64(0.0), np.float64(0.02), np.float64(0.05)]
서울: 전체=4383 | 결측수=[np.int64(26), np.int64(26), np.int64(29)] | 결측률=[np.float64(0.59), np.float64(0.59), np.float64(0.66)]
수원: 전체=4383 | 결측수=[np.int64(3), np.int64(3), np.int64(2)] | 결측률=[np.float64(0.07), np.float64(0.07), np.float64(0.05)]
원주: 전체=4383 | 결측수=[np.int64(6), np.int64(6), np.int64(6)] | 결측률=[np.float64(0.14), np.float64(0.14), np.flo

In [ ]:
import pandas as pd
import os

def add_RLG_column (file_to_process,):
    chunk_list = [] 
    print(f"Reading and processing '{file_to_process}' in chunks...")
    try:
        # Create a list to hold processed chunks
        # 처리된 청크를 저장할 리스트 생성
        processed_chunks = []
        
        # Read the CSV in chunks
        # CSV를 청크 단위로 읽기
        for chunk in pd.read_csv(file_to_process, encoding='cp949', chunksize=10000, low_memory=False):
            # Add the '광역자치단체' (metropolitan area) column by mapping the '지점명' (branch name)
            # '지점명'을 매핑하여 '광역자치단체' 컬럼 추가
            chunk['광역자치단체'] = chunk['지점명'].map(region_map)
            
            # Append the processed chunk to the list
            # 처리된 청크를 리스트에 추가
            processed_chunks.append(chunk)

        # Concatenate all processed chunks into a single DataFrame
        # 처리된 모든 청크를 하나의 데이터프레임으로 연결
        if processed_chunks:
            df_final_with_region = pd.concat(processed_chunks, ignore_index=True)

            # Display information about the updated DataFrame
            # 업데이트된 데이터프레임 정보 출력
            print("\n--- Updated DataFrame Info ---")
            print("Shape of the final DataFrame:", df_final_with_region.shape)
            print("\nFirst 5 rows with the new column:")
            print(df_final_with_region[['지점명', '광역자치단체']].head())
            print("\nValue counts of the new '광역자치단체' column:")
            print(df_final_with_region['광역자치단체'].value_counts())

            # Save the updated DataFrame back to the CSV file
            # 업데이트된 데이터프레임을 CSV 파일로 다시 저장
            print(f"\nSaving the updated data back to '{file_to_process}'...")
            df_final_with_region.to_csv(file_to_process, index=False, encoding='cp949')
            print("Successfully saved the updated file.")
        else:
            print(f"No data was processed from '{file_to_process}'. The file might be empty.")

    except FileNotFoundError:
        print(f"Error: The file was not found at '{file_to_process}'")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
import pandas as pd
import os

# Define the path to the filtered CSV file
# The user specified 'ASOS_통합(1301~2412)_filtered.csv'
file_to_process = os.path.join(os.path.join(asos_folder_path, '가공후'), 'ASOS_통합(1301~2412)_filtered.csv')

# Define the mapping from branch name to metropolitan area
# 지점명을 광역자치단체로 매핑하는 딕셔너리 정의
region_map = {
    '서울': '서울특별시',
    '부산': '부산광역시',
    '북부산': '부산광역시',
    '대구': '대구광역시',
    '대구(기)': '대구광역시',
    '인천': '인천광역시',
    '광주': '광주광역시',
    '대전': '대전광역시',
    '수원': '경기도',
    '강릉': '강원특별자치도',
    '대관령': '강원특별자치도',
    '북강릉': '강원특별자치도',
    '북춘천': '강원특별자치도',
    '춘천': '강원특별자치도',
    '원주': '강원특별자치도',
    '청주': '충청북도',
    '추풍령': '충청북도',
    '서청주': '충청북도',
    '서산': '충청남도',
    '홍성': '충청남도',
    '전주': '전북특별자치도',
    '고창': '전북특별자치도',
    '고창군': '전북특별자치도',
    '순창군': '전북특별자치도',
    '목포': '전라남도',
    '흑산도': '전라남도',
    '영광군': '전라남도',
    '강진군': '전라남도',
    '보성군': '전라남도',
    '광양시': '전라남도',
    '포항': '경상북도',
    '안동': '경상북도',
    '경주시': '경상북도',
    '청송군': '경상북도',
    '김해시': '경상남도',
    '양산시': '경상남도',
    '의령군': '경상남도',
    '함양군': '경상남도',
    '북창원': '경상남도',
    '진주': '경상남도',
    '고산': '제주특별자치도',
    '제주': '제주특별자치도'
}

# Process the file in chunks
# 파일을 청크 단위로 처리
chunk_list = []
print(f"Reading and processing '{file_to_process}' in chunks...")
try:
    # Create a list to hold processed chunks
    # 처리된 청크를 저장할 리스트 생성
    processed_chunks = []
    
    # Read the CSV in chunks
    # CSV를 청크 단위로 읽기
    for chunk in pd.read_csv(file_to_process, encoding='cp949', chunksize=10000, low_memory=False):
        # Add the '광역자치단체' (metropolitan area) column by mapping the '지점명' (branch name)
        # '지점명'을 매핑하여 '광역자치단체' 컬럼 추가
        chunk['광역자치단체'] = chunk['지점명'].map(region_map)
        
        # Append the processed chunk to the list
        # 처리된 청크를 리스트에 추가
        processed_chunks.append(chunk)

    # Concatenate all processed chunks into a single DataFrame
    # 처리된 모든 청크를 하나의 데이터프레임으로 연결
    if processed_chunks:
        df_final_with_region = pd.concat(processed_chunks, ignore_index=True)

        # Display information about the updated DataFrame
        # 업데이트된 데이터프레임 정보 출력
        print("\n--- Updated DataFrame Info ---")
        print("Shape of the final DataFrame:", df_final_with_region.shape)
        print("\nFirst 5 rows with the new column:")
        print(df_final_with_region[['지점명', '광역자치단체']].head())
        print("\nValue counts of the new '광역자치단체' column:")
        print(df_final_with_region['광역자치단체'].value_counts())

        # Save the updated DataFrame back to the CSV file
        # 업데이트된 데이터프레임을 CSV 파일로 다시 저장
        print(f"\nSaving the updated data back to '{file_to_process}'...")
        df_final_with_region.to_csv(file_to_process, index=False, encoding='cp949')
        print("Successfully saved the updated file.")
    else:
        print(f"No data was processed from '{file_to_process}'. The file might be empty.")

except FileNotFoundError:
    print(f"Error: The file was not found at '{file_to_process}'")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
import pandas as pd
import os

# Define the path to the filtered CSV file
# The user specified 'ASOS_통합(2001~2412)_filtered.csv'
file_to_process = os.path.join(os.path.join(asos_folder_path, '가공후'), 'ASOS_통합(2001~2412)_filtered.csv')

# Define the mapping from branch name to metropolitan area
# 지점명을 광역자치단체로 매핑하는 딕셔너리 정의
region_map = {
    '서울': '서울특별시',
    '부산': '부산광역시',
    '북부산': '부산광역시',
    '대구': '대구광역시',
    '대구(기)': '대구광역시',
    '인천': '인천광역시',
    '광주': '광주광역시',
    '대전': '대전광역시',
    '수원': '경기도',
    '강릉': '강원특별자치도',
    '대관령': '강원특별자치도',
    '북강릉': '강원특별자치도',
    '북춘천': '강원특별자치도',
    '춘천': '강원특별자치도',
    '원주': '강원특별자치도',
    '청주': '충청북도',
    '추풍령': '충청북도',
    '서청주': '충청북도',
    '서산': '충청남도',
    '홍성': '충청남도',
    '전주': '전북특별자치도',
    '고창': '전북특별자치도',
    '고창군': '전북특별자치도',
    '순창군': '전북특별자치도',
    '목포': '전라남도',
    '흑산도': '전라남도',
    '영광군': '전라남도',
    '강진군': '전라남도',
    '보성군': '전라남도',
    '광양시': '전라남도',
    '포항': '경상북도',
    '안동': '경상북도',
    '경주시': '경상북도',
    '청송군': '경상북도',
    '김해시': '경상남도',
    '양산시': '경상남도',
    '의령군': '경상남도',
    '함양군': '경상남도',
    '북창원': '경상남도',
    '진주': '경상남도',
    '고산': '제주특별자치도',
    '제주': '제주특별자치도'
}

# Process the file in chunks
# 파일을 청크 단위로 처리
print(f"Reading and processing '{file_to_process}' in chunks...")
try:
    # Create a list to hold processed chunks
    # 처리된 청크를 저장할 리스트 생성
    processed_chunks = []
    
    # Read the CSV in chunks
    # CSV를 청크 단위로 읽기
    for chunk in pd.read_csv(file_to_process, encoding='cp949', chunksize=10000, low_memory=False):
        # Add the '광역자치단체' (metropolitan area) column by mapping the '지점명' (branch name)
        # '지점명'을 매핑하여 '광역자치단체' 컬럼 추가
        chunk['광역자치단체'] = chunk['지점명'].map(region_map)
        
        # Append the processed chunk to the list
        # 처리된 청크를 리스트에 추가
        processed_chunks.append(chunk)

    # Concatenate all processed chunks into a single DataFrame
    # 처리된 모든 청크를 하나의 데이터프레임으로 연결
    if processed_chunks:
        df_final_with_region = pd.concat(processed_chunks, ignore_index=True)

        # Display information about the updated DataFrame
        # 업데이트된 데이터프레임 정보 출력
        print("\n--- Updated DataFrame Info ---")
        print("Shape of the final DataFrame:", df_final_with_region.shape)
        print("\nFirst 5 rows with the new column:")
        print(df_final_with_region[['지점명', '광역자치단체']].head())
        print("\nValue counts of the new '광역자치단체' column:")
        print(df_final_with_region['광역자치단체'].value_counts())

        # Save the updated DataFrame back to the CSV file
        # 업데이트된 데이터프레임을 CSV 파일로 다시 저장
        print(f"\nSaving the updated data back to '{file_to_process}'...")
        df_final_with_region.to_csv(file_to_process, index=False, encoding='cp949')
        print("Successfully saved the updated file.")
    else:
        print(f"No data was processed from '{file_to_process}'. The file might be empty.")

except FileNotFoundError:
    print(f"Error: The file was not found at '{file_to_process}'")
except Exception as e:
    print(f"An error occurred: {e}")